In [1]:
from keras.preprocessing.image import ImageDataGenerator
import model_util as util
from keras.applications import VGG19
import os

base_dir = '/home/jupyter/Data'
train_dir = os.path.join(base_dir, 'train')

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      featurewise_center=True,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
    validation_split=0.2
    )



train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        subset="training",
        shuffle = True
        )

validation_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        subset="validation",
        shuffle = True
        )

Using TensorFlow backend.


Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [8]:
from keras import optimizers
p = {'lr': (1e-3, 1e-5),# ],
     #'batch_size': (10,20, 30),
     'epochs': [150],

     'optimizer': [optimizers.Adam,optimizers.RMSprop],
     #'losses': ['logcosh', 'binary_crossentropy'],
     'activation':['relu','elu'],
     #'last_activation': ['sigmoid']
    }
    

In [9]:
models = []
for lr in p['lr']:
    for epoch in p['epochs']:
        #for ls in p['losses']:
            #for bs in p['batch_size']:
                for opt in p['optimizer']:
                    for act in p['activation']:
                        #print(str(lr) + ' ' + str(epoch) + ' ' + ls + ' ' + str(bs) + ' ' + str(opt) + ' '+ act)
                        runmodel = [lr,epoch,opt,act]
                        models.append(runmodel)

                

In [10]:
for i in models:
    print(i)

[0.001, 150, <class 'keras.optimizers.Adam'>, 'relu']
[0.001, 150, <class 'keras.optimizers.Adam'>, 'elu']
[0.001, 150, <class 'keras.optimizers.RMSprop'>, 'relu']
[0.001, 150, <class 'keras.optimizers.RMSprop'>, 'elu']
[1e-05, 150, <class 'keras.optimizers.Adam'>, 'relu']
[1e-05, 150, <class 'keras.optimizers.Adam'>, 'elu']
[1e-05, 150, <class 'keras.optimizers.RMSprop'>, 'relu']
[1e-05, 150, <class 'keras.optimizers.RMSprop'>, 'elu']


In [5]:


vgg19_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

vgg19_base.trainable = True

set_trainable = False
for layer in vgg19_base.layers:
    if layer.name == 'block5_conv1' or layer.name == 'block4_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
print(len(vgg19_base.trainable_weights))

16


In [6]:
#for i in models
?util.run_tuning_model_2hidden_hyper


Signature:
util.run_tuning_model_2hidden_hyper(
    ['pretraind_model', 'lrate', 'num_epoch', 'optimizers', 'activate_func', 'train_generator', 'validation_generator', 'final_models', 'count'],
)
Docstring: <no docstring>
File:      ~/ML1020/model_util.py
Type:      function


In [7]:
final_models = []
count = 0
for i in models:
    count += 1
    output = util.run_tuning_model_2hidden_hyper(vgg19_base,i[0],i[1],i[2],i[3],train_generator,validation_generator,final_models,count)

Started
Epoch 1/1
10/10 [==============================] - 9s 884ms/step - loss: 2.6831 - acc: 0.0750 - val_loss: 2.3187 - val_acc: 0.0800
0.001
Finished
Saved model to disk
[['Model_ 1 ', 0.001, 1, <class 'keras.optimizers.Adam'>, 'relu', 0.07500000186264515, 2.683117651939392, 0.0800000011920929, 2.3187397956848144]]
Started
Epoch 1/2
10/10 [==============================] - 6s 584ms/step - loss: 2.3329 - acc: 0.1100 - val_loss: 2.2992 - val_acc: 0.1300
0.001
Epoch 2/2
10/10 [==============================] - 3s 287ms/step - loss: 2.3035 - acc: 0.0850 - val_loss: 2.3012 - val_acc: 0.1150
0.001
Finished
Saved model to disk
[['Model_ 1 ', 0.001, 1, <class 'keras.optimizers.Adam'>, 'relu', 0.07500000186264515, 2.683117651939392, 0.0800000011920929, 2.3187397956848144], ['Model_ 2 ', 0.001, 2, <class 'keras.optimizers.Adam'>, 'relu', 0.08500000163912773, 2.3035458087921143, 0.11500000171363353, 2.3011946201324465]]
Started
Epoch 1/1
10/10 [==============================] - 6s 586ms/step 

In [19]:
import pandas as pd
df = pd.DataFrame(output,columns = ['Model','Inital Learning Rate','Max Epochs','Optimizers','Activation','Train Accuracy','Train Loss','Val Accuracy','Val Loss'])
df.to_csv('/home/jupyter/TuningOutput/TuningOutput.csv',index=False)